# CCD theory for a closed-shell reference

In this notebook we will use wicked to generate equations for the CCD method

In [1]:
import wicked as w

In [2]:
w.reset_space()
w.add_space('o', 'fermion', 'occupied', ['i', 'j', 'k', 'l', 'm', 'n'])
w.add_space('v', 'fermion', 'unoccupied', ['a', 'b', 'c', 'd', 'e', 'f'])

T2 = w.op('T2', ['v+ v+ o o'])
F = w.op('F', ['o+ o', 'v+ v','o+ v', 'v+ o'])
V = w.op(
    'V',
    [
        'o+ o+ o o',
        'o+ o+ v o',
        'o+ o+ v v',
        'o+ v+ o o',
        'o+ v+ v o',
        'o+ v+ v v',
        'v+ v+ o o',
        'v+ v+ v o',
        'v+ v+ v v',
    ],
)

In [3]:
wt = w.WickTheorem()

expr = wt.contract(w.rational(1), F, 0, 4)
expr += wt.contract(w.rational(1), V, 0, 4)
expr += wt.contract(w.rational(1), w.commutator(F, T2), 0, 4)
expr += wt.contract(w.rational(1), w.commutator(V, T2), 0, 4)
expr += wt.contract(w.rational(1,2), w.commutator(w.commutator(F,T2),T2), 0, 4)
expr += wt.contract(w.rational(1,2), w.commutator(w.commutator(V, T2),T2), 0, 4)
mbeq = expr.to_manybody_equation("R")

code = ["def evaluate_residual(F,V,T):",
        "    # contributions to the residual",
        "    Roovv = np.zeros((nocc,nocc,nvir,nvir))"]
for eq in mbeq[4]:
    contraction = eq.compile("einsum")
    if 'Roovv' in contraction:
        code.append(f'    {contraction}')
code.append(f'    return Roovv')
funct = '\n'.join(code)
    
print(f'Defined the function:\n\n{funct}\n')
exec(funct)

Defined the function:

def evaluate_residual(F,V,T):
    # contributions to the residual
    Roovv = np.zeros((nocc,nocc,nvir,nvir))
    Roovv += -2.000000000 * np.einsum("ca,ijbc->ijab",F["vv"],T2["oovv"])
    Roovv += 2.000000000 * np.einsum("ik,jkab->ijab",F["oo"],T2["oovv"])
    Roovv += 0.500000000 * np.einsum("ijcd,cdab->ijab",T2["oovv"],V["vvvv"])
    Roovv += 2.000000000 * np.einsum("ikac,jlbd,cdkl->ijab",T2["oovv"],T2["oovv"],V["vvoo"])
    Roovv += -4.000000000 * np.einsum("ikac,jckb->ijab",T2["oovv"],V["ovov"])
    Roovv += -1.000000000 * np.einsum("ijac,klbd,cdkl->ijab",T2["oovv"],T2["oovv"],V["vvoo"])
    Roovv += 0.250000000 * np.einsum("klab,ijcd,cdkl->ijab",T2["oovv"],T2["oovv"],V["vvoo"])
    Roovv += 0.500000000 * np.einsum("klab,ijkl->ijab",T2["oovv"],V["oooo"])
    Roovv += -1.000000000 * np.einsum("ikab,jlcd,cdkl->ijab",T2["oovv"],T2["oovv"],V["vvoo"])
    Roovv += 1.000000000 * np.einsum("ijab->ijab",V["oovv"])
    return Roovv



```python
def evaluate_residual(F,V,T):
    # contributions to the residual
    Roovv = np.zeros((nocc,nocc,nvir,nvir))
    Roovv += -2.000000000 * np.einsum("ca,ijbc->ijab",F["vv"],T2["oovv"])
    Roovv += 2.000000000 * np.einsum("ik,jkab->ijab",F["oo"],T2["oovv"])
    Roovv += 0.500000000 * np.einsum("ijcd,cdab->ijab",T2["oovv"],V["vvvv"])
    Roovv += 2.000000000 * np.einsum("ikac,jlbd,cdkl->ijab",T2["oovv"],T2["oovv"],V["vvoo"])
    Roovv += -4.000000000 * np.einsum("ikac,jckb->ijab",T2["oovv"],V["ovov"])
    Roovv += -1.000000000 * np.einsum("ijac,klbd,cdkl->ijab",T2["oovv"],T2["oovv"],V["vvoo"])
    Roovv += 0.250000000 * np.einsum("klab,ijcd,cdkl->ijab",T2["oovv"],T2["oovv"],V["vvoo"])
    Roovv += 0.500000000 * np.einsum("klab,ijkl->ijab",T2["oovv"],V["oooo"])
    Roovv += -1.000000000 * np.einsum("ikab,jlcd,cdkl->ijab",T2["oovv"],T2["oovv"],V["vvoo"])
    Roovv += 1.000000000 * np.einsum("ijab->ijab",V["oovv"])
    return Roovv
```

In [4]:
# def spin_mask2(i,j):
#     if (i % 2) == (j % 2):
#         return 1
#     return 0

# def spin_mask4(i,j):
#     if (i % 2) == (j % 2):
#         return 1
#     return 0

# np.fromfunction(np.vectorize(spin_mask), (4, 4), dtype=int)

In [5]:
import psi4
import forte
import forte.utils
from forte import forte_options
import numpy as np

## Compute the Hartree–Fock and MP2 energy

In [6]:
# setup xyz geometry for linear H4
geometry = """
H 0.0 0.0 0.0
H 0.0 0.0 1.0
H 0.0 0.0 2.0
H 0.0 0.0 3.0
H 0.0 0.0 4.0
H 0.0 0.0 5.1
symmetry c1
"""

(Escf, psi4_wfn) = forte.utils.psi4_scf(geometry,
                                        basis='sto-3g',
                                        reference='rhf',
                                        options={'E_CONVERGENCE' : 1.e-12})

## Prepare integrals for Forte

In [7]:
# Define the orbital spaces
mo_spaces = {'RESTRICTED_DOCC': [3],'RESTRICTED_UOCC': [3]}

# pass Psi4 options to Forte
options = psi4.core.get_options()
options.set_current_module('FORTE')
forte_options.get_options_from_psi4(options)

# Grab the number of MOs per irrep
nmopi = psi4_wfn.nmopi()
# Grab the point group symbol (e.g. "C2V")
point_group = psi4_wfn.molecule().point_group().symbol()
# create a MOSpaceInfo object
mo_space_info = forte.make_mo_space_info_from_map(nmopi, point_group,mo_spaces, [])
# make a ForteIntegral object
ints = forte.make_ints_from_psi4(psi4_wfn, forte_options, mo_space_info)

## Define orbital spaces and dimensions

In [8]:
occmos = mo_space_info.corr_absolute_mo('RESTRICTED_DOCC')
virmos = mo_space_info.corr_absolute_mo('RESTRICTED_UOCC')
allmos = mo_space_info.corr_absolute_mo('CORRELATED')
nocc = 2 * len(occmos)
nvir = 2 * len(virmos)

## Build the Fock matrix and the zeroth-order Fock matrix

In [9]:
# Build the Fock matrix blocks
F = {'oo': forte.spinorbital_fock(ints,occmos, occmos,occmos),
     'vv': forte.spinorbital_fock(ints,virmos, virmos,occmos),
     'ov': forte.spinorbital_fock(ints,occmos, virmos,occmos)}

# Build the diagonal orbital energies
Fdiag = {'oo': np.diag(F['oo']), 'vv': np.diag(F['vv'])}
F0 = {'oo' : np.diag(Fdiag['oo']),'vv' : np.diag(Fdiag['vv']) }

# Build the two-electron integrals
V = {}
V["oovv"] = forte.spinorbital_tei(ints,occmos,occmos,virmos,virmos)
V["vvvv"] = forte.spinorbital_tei(ints,virmos,virmos,virmos,virmos)
V["vvoo"] = forte.spinorbital_tei(ints,virmos,virmos,occmos,occmos)
V["ovov"] = forte.spinorbital_tei(ints,occmos,virmos,occmos,virmos)
V["oooo"] = forte.spinorbital_tei(ints,occmos,occmos,occmos,occmos)

## Build the MP denominators

In [10]:
d2 = np.zeros((nocc,nocc,nvir,nvir))

fo = Fdiag['oo']
fv = Fdiag['vv']
for i in range(nocc):
    for j in range(nocc):
        for a in range(nvir):
            for b in range(nvir):
                si = i % 2
                sj = j % 2
                sa = a % 2
                sb = b % 2
                if si == sj == sa == sb:
                    d2[i][j][a][b] = 1.0 / (fo[i] + fo[j] - fv[a] - fv[b])
                if si == sa and sj == sb and si != sj:
                    d2[i][j][a][b] = 1.0 / (fo[i] + fo[j] - fv[a] - fv[b])
                if si == sb and sj == sa and si != sj:
                    d2[i][j][a][b] = 1.0 / (fo[i] + fo[j] - fv[a] - fv[b])

In [11]:
# Compute the MP2 correlation energy
Emp2 = 0.0
for i in range(nocc):
    for j in range(nocc):
        for a in range(nvir):
            for b in range(nvir):
                Emp2 += 0.25 * V["oovv"][i][j][a][b] ** 2 / (fo[i] + fo[j] - fv[a] - fv[b])
print(f"MP2 corr. energy: {Emp2:.12f} Eh")

MP2 corr. energy: -0.066236921093 Eh


In [12]:
def antisymmetrize_residual(Roovv):
    # antisymmetrize the residual
    Roovv_anti = np.zeros((nocc,nocc,nvir,nvir))
    Roovv_anti += 0.25 * np.einsum("ijab->ijab",Roovv)
    Roovv_anti -= 0.25 * np.einsum("ijab->jiab",Roovv)
    Roovv_anti -= 0.25 * np.einsum("ijab->ijba",Roovv)
    Roovv_anti += 0.25 * np.einsum("ijab->jiba",Roovv)    
    return Roovv_anti

def update_amplitudes(T2,R,d2):
    T2["oovv"] += np.einsum("ijab,ijab->ijab",R,d2)
    
def compute_energy(T2,V):
    energy = 0.25 * np.einsum('ijab,ijab->', V["oovv"], T2["oovv"])
    return energy

In [13]:
T2 = {}
T2["oovv"] = np.zeros((nocc,nocc,nvir,nvir))

maxiter = 40
for i in range(maxiter):
    Roovv = evaluate_residual(F,V,T2)
    Roovv = antisymmetrize_residual(Roovv)
    update_amplitudes(T2,Roovv,d2)
    Ewicked = compute_energy(T2,V)
    
    # check for convergence
    norm_R = np.linalg.norm(Roovv)
    print(f"{i:3d} {Ewicked:+.12f} {norm_R:e}") 
    
    if norm_R < 1.0e-9:
        break
    
# ref_CCD = -0.067744692922 # from psi4numpy (H4)
ref_CCD = -0.107288757159 # from psi4numpy (H6)
print(f"CCD corr. energy: {Ewicked:+.12f} Eh")
print(f"Err corr. energy: {Ewicked - ref_CCD:+.12e} Eh")

  0 -0.066236921093 7.023516e-01
  1 -0.088946060413 2.703142e-01
  2 -0.098696464425 1.216200e-01
  3 -0.103031524033 6.068921e-02
  4 -0.105094976371 3.242887e-02
  5 -0.106116982998 1.810818e-02
  6 -0.106647059295 1.035930e-02
  7 -0.106930043777 6.012057e-03
  8 -0.107085395675 3.519183e-03
  9 -0.107172178276 2.072075e-03
 10 -0.107221402582 1.224934e-03
 11 -0.107249599939 7.262866e-04
 12 -0.107265887451 4.315530e-04
 13 -0.107275349928 2.568358e-04
 14 -0.107280874030 1.530328e-04
 15 -0.107284110660 9.126199e-05
 16 -0.107286012879 5.445897e-05
 17 -0.107287133582 3.251250e-05
 18 -0.107287795236 1.941678e-05
 19 -0.107288186548 1.159872e-05
 20 -0.107288418322 6.929780e-06
 21 -0.107288555776 4.140796e-06
 22 -0.107288637381 2.474500e-06
 23 -0.107288685876 1.478833e-06
 24 -0.107288714717 8.838329e-07
 25 -0.107288731881 5.282445e-07
 26 -0.107288742102 3.157251e-07
 27 -0.107288748192 1.887077e-07
 28 -0.107288751823 1.127910e-07
 29 -0.107288753987 6.741579e-08
 30 -0.107